In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from concurrent.futures import ThreadPoolExecutor

# 定义常数
R = 0.3  # 金属板半径，单位：米
D = 0.001  # 金属板厚度，单位：米
V0 = 100  # 接电压的金属板电压，单位：伏
epsilon0 = 8.85e-12  # 真空介电常数，单位：法拉/米

# 定义网格参数
nx, ny, nz = 100, 100, 100  # 网格点数
dx, dy, dz = 2*R/nx, 2*R/ny, 2*R/nz  # 网格间距

# 定义电势数组和电荷密度数组
phi = np.zeros((nx, ny, nz))
rho = np.zeros((nx, ny, nz))

# 定义金属板位置
z1 = int(nz/2 - D/(2*dz))
z2 = int(nz/2 + D/(2*dz))

# 设置边界条件
phi[:, :, z1] = 0  # 接地金属板
phi[:, :, z2] = V0  # 接电压金属板

# 定义泊松方程的系数矩阵
def poisson_matrix(nx, ny, nz, dx, dy, dz):
    n = nx * ny * nz
    data = []
    row = []
    col = []

    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                index = i*ny*nz + j*nz + k
                if i > 0:
                    data.append(1/dx**2)
                    row.append(index)
                    col.append(index - ny*nz)
                if i < nx-1:
                    data.append(1/dx**2)
                    row.append(index)
                    col.append(index + ny*nz)
                if j > 0:
                    data.append(1/dy**2)
                    row.append(index)
                    col.append(index - nz)
                if j < ny-1:
                    data.append(1/dy**2)
                    row.append(index)
                    col.append(index + nz)
                if k > 0:
                    data.append(1/dz**2)
                    row.append(index)
                    col.append(index - 1)
                if k < nz-1:
                    data.append(1/dz**2)
                    row.append(index)
                    col.append(index + 1)
                data.append(-2/dx**2 - 2/dy**2 - 2/dz**2)
                row.append(index)
                col.append(index)

    A = csr_matrix((data, (row, col)), shape=(n, n))
    return A

# 求解泊松方程
def solve_poisson(A, rho, phi):
    rhs = -rho.ravel() / epsilon0
    phi_flat = spsolve(A, rhs)
    phi[:] = phi_flat.reshape(phi.shape)

# 计算电场
def compute_efield(phi, dx, dy, dz):
    ex = np.zeros_like(phi)
    ey = np.zeros_like(phi)
    ez = np.zeros_like(phi)

    ex[:, :, 1:-1] = (phi[:, :, :-2] - phi[:, :, 2:]) / (2*dx)
    ey[:, 1:-1, :] = (phi[:, :-2, :] - phi[:, 2:, :]) / (2*dy)
    ez[1:-1, :, :] = (phi[:-2, :, :] - phi[2:, :, :]) / (2*dz)

    return ex, ey, ez

# 多线程计算电荷密度
def compute_charge_density(rho, z1, z2):
    def worker(i, j):
        rho[i, j, z1] = -epsilon0 * (phi[i, j, z1-1] - phi[i, j, z1]) / dz
        rho[i, j, z2] = epsilon0 * (phi[i, j, z2] - phi[i, j, z2+1]) / dz

    with ThreadPoolExecutor() as executor:
        for i in range(nx):
            for j in range(ny):
                executor.submit(worker, i, j)

# 主程序
A = poisson_matrix(nx, ny, nz, dx, dy, dz)

for _ in range(1000):  # 迭代求解
    solve_poisson(A, rho, phi)
    compute_charge_density(rho, z1, z2)

ex, ey, ez = compute_efield(phi, dx, dy, dz)

# 输出结果
print("Electric potential:")
print(phi)
print("Electric field:")
print(ex)
print(ey)
print(ez)
print("Charge density:")
print(rho)